In [8]:
# Given an m x n integer matrix heightMap representing the height of each unit cell in a 2D elevation map,
# return the volume of water it can trap after raining.
# Input: heightMap = [[1,4,3,1,3,2],[3,2,1,3,2,4],[2,3,3,2,3,1]]
# Output: 4
# Explanation: After the rain, water is trapped between the blocks.
# We have two small ponds 1 and 3 units trapped.
# The total volume of water trapped is 4.
import heapq
from typing import List


In [9]:
class Solution:
    def trapRainWater(self, heightMap: List[List[int]]) -> int:
        # Edge case: if the heightMap is empty or has no columns
        if not heightMap or not heightMap[0]:
            return 0
        # Initialize dimensions, visited matrix, and min-heap
        m, n = len(heightMap), len(heightMap[0])
        visited = [[False] * n for _ in range(m)]
        heap = []
        
        # Add all the boundary cells to the heap and mark them as visited
        for i in range(m): # Iterate through each row
            for j in range(n): # Iterate through each column
                if i == 0 or i == m - 1 or j == 0 or j == n - 1: # Check if the cell is on the boundary
                    heapq.heappush(heap, (heightMap[i][j], i, j)) # Push the cell height and its coordinates into the heap
                    visited[i][j] = True # Mark the cell as visited

        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)] # Possible directions to move in the grid
        water_trapped = 0 # Initialize water trapped counter
        max_height = float('-inf') # Initialize max height to negative infinity

        while heap: # While there are cells in the heap
            height, x, y = heapq.heappop(heap) # Pop the cell with the smallest height
            max_height = max(max_height, height) # Update the max height encountered so far
 
            for dx, dy in directions: # Explore all four possible directions
                nx, ny = x + dx, y + dy # Calculate the new coordinates
                if 0 <= nx < m and 0 <= ny < n and not visited[nx][ny]: 
                    visited[nx][ny] = True # Mark the new cell as visited
                    if heightMap[nx][ny] < max_height:  # If the new cell is lower than the max height
                        water_trapped += max_height - heightMap[nx][ny] # Calculate the water trapped
                    heapq.heappush(heap, (heightMap[nx][ny], nx, ny)) # Push the new cell into the heap

        return water_trapped # Return the total water trapped

In [10]:
Solution().trapRainWater(heightMap = [[1,4,3,1,3,2],[3,2,1,3,2,4],[2,3,3,2,3,1]])

4

## 📊 Understanding heightMap

`heightMap` is a 2D matrix where:
- **Rows (m)**: number of rows in the grid
- **Columns (n)**: number of columns in the grid
- **Each value**: the height/elevation of that cell

### Example:
```
heightMap = [[1,4,3,1,3,2],
             [3,2,1,3,2,4],
             [2,3,3,2,3,1]]
```

- `m = 3` (3 rows)
- `n = 6` (6 columns)
- `heightMap[0][1] = 4` means cell at position (row=0, col=1) has height 4
- `heightMap[1][2] = 1` means cell at position (row=1, col=2) has height 1 (a valley!)


In [1]:
# Let's visualize the heightMap
heightMap = [[1,4,3,1,3,2],
             [3,2,1,3,2,4],
             [2,3,3,2,3,1]]

print("Grid View (Height Values):")
print("=" * 40)
for i, row in enumerate(heightMap):
    print(f"Row {i}: {row}")

print("\n" + "=" * 40)
print(f"Dimensions: {len(heightMap)} rows × {len(heightMap[0])} columns")
print("=" * 40)

# Show specific cells
print("\nExample cell access:")
print(f"heightMap[0][1] = {heightMap[0][1]} (row 0, col 1)")
print(f"heightMap[1][2] = {heightMap[1][2]} (row 1, col 2) ← Low point!")
print(f"heightMap[1][5] = {heightMap[1][5]} (row 1, col 5)")

Grid View (Height Values):
Row 0: [1, 4, 3, 1, 3, 2]
Row 1: [3, 2, 1, 3, 2, 4]
Row 2: [2, 3, 3, 2, 3, 1]

Dimensions: 3 rows × 6 columns

Example cell access:
heightMap[0][1] = 4 (row 0, col 1)
heightMap[1][2] = 1 (row 1, col 2) ← Low point!
heightMap[1][5] = 4 (row 1, col 5)


## 🎯 How We Find "Surrounded" Points

### Strategy: Work from OUTSIDE → INSIDE

Instead of searching for surrounded points directly, we:
1. **Start at the BOUNDARIES** (edges cannot hold water - it flows out!)
2. **Process the LOWEST boundary first** (using a min-heap)
3. **Move INWARD** step by step
4. **Track the maximum height** we've seen (this is the "water level")
5. **If we find a cell LOWER than the water level** → It's surrounded and traps water! 💧

### Why This Works:
- **Boundaries** = escape routes for water
- **Lowest boundary** = weakest point where water escapes
- Any cell **lower than the lowest surrounding wall** will trap water up to that wall height

In [2]:
# Visual demonstration: Which cells are boundaries vs inner cells?
heightMap = [[1,4,3,1,3,2],
             [3,2,1,3,2,4],
             [2,3,3,2,3,1]]

m, n = len(heightMap), len(heightMap[0])

print("🔷 BOUNDARY CELLS (starting points - water can flow out here):")
print("=" * 60)
boundary_cells = []
for i in range(m):
    for j in range(n):
        if i == 0 or i == m-1 or j == 0 or j == n-1:
            boundary_cells.append((i, j, heightMap[i][j]))
            print(f"  Position ({i},{j}): height = {heightMap[i][j]}")

print("\n" + "=" * 60)
print(f"Total boundary cells: {len(boundary_cells)}")

print("\n🔶 INNER CELLS (potentially surrounded - may trap water):")
print("=" * 60)
inner_cells = []
for i in range(m):
    for j in range(n):
        if not (i == 0 or i == m-1 or j == 0 or j == n-1):
            inner_cells.append((i, j, heightMap[i][j]))
            print(f"  Position ({i},{j}): height = {heightMap[i][j]}")

print("\n" + "=" * 60)
print(f"Total inner cells: {len(inner_cells)}")
print("\n💡 We check these inner cells to see if they trap water!")

🔷 BOUNDARY CELLS (starting points - water can flow out here):
  Position (0,0): height = 1
  Position (0,1): height = 4
  Position (0,2): height = 3
  Position (0,3): height = 1
  Position (0,4): height = 3
  Position (0,5): height = 2
  Position (1,0): height = 3
  Position (1,5): height = 4
  Position (2,0): height = 2
  Position (2,1): height = 3
  Position (2,2): height = 3
  Position (2,3): height = 2
  Position (2,4): height = 3
  Position (2,5): height = 1

Total boundary cells: 14

🔶 INNER CELLS (potentially surrounded - may trap water):
  Position (1,1): height = 2
  Position (1,2): height = 1
  Position (1,3): height = 3
  Position (1,4): height = 2

Total inner cells: 4

💡 We check these inner cells to see if they trap water!


## 📝 Step-by-Step Algorithm Explanation

### The Process:

**Step 1: Initialize**
- Put ALL 14 boundary cells in a min-heap (sorted by height)
- Mark them as visited
- These are our "walls" - water can escape through them

**Step 2: Process Lowest Boundary First**
- Pop the cell with **minimum height** from heap
- This is the weakest point where water can escape
- Update `max_height` (our water level)

**Step 3: Explore Neighbors**
- Look at the 4 neighbors (up, down, left, right)
- If neighbor is **unvisited**:
  - If `neighbor_height < max_height` → **WATER TRAPPED!** 💧
  - Water amount = `max_height - neighbor_height`
  - Add neighbor to heap (it's now a new boundary)

**Step 4: Repeat**
- Keep processing until all cells are visited
- We gradually move from outside to inside, finding all trapped water

### 🔑 Key Concept:
The `max_height` represents the **minimum barrier** needed to escape. Any cell lower than this gets filled with water!

## 🔷 What are Boundary Cells?

**Boundary cells** = All cells on the **outer edges** of the grid

### Why are they important?
- Water on the boundary can **always flow out** (no walls to contain it)
- They act as the **starting walls** for trapping water
- We start from these cells and work inward

### How to identify boundary cells:
A cell at position `(i, j)` is a boundary if:
- `i == 0` (top row) **OR**
- `i == m-1` (bottom row) **OR**
- `j == 0` (left column) **OR**
- `j == n-1` (right column)

In [3]:
# Visual map showing EXACTLY which cells are boundaries
heightMap = [[1,4,3,1,3,2],
             [3,2,1,3,2,4],
             [2,3,3,2,3,1]]

m, n = len(heightMap), len(heightMap[0])

print("📍 GRID WITH BOUNDARY MARKERS:")
print("=" * 70)
print("Legend: [B] = Boundary cell,  [I] = Inner cell\n")

for i in range(m):
    row_display = []
    for j in range(n):
        is_boundary = (i == 0 or i == m-1 or j == 0 or j == n-1)
        marker = "[B]" if is_boundary else "[I]"
        row_display.append(f"{marker}{heightMap[i][j]}")
    print(f"Row {i}: " + "  ".join(row_display))

print("\n" + "=" * 70)

# Count them
boundary_count = 0
inner_count = 0
for i in range(m):
    for j in range(n):
        if i == 0 or i == m-1 or j == 0 or j == n-1:
            boundary_count += 1
        else:
            inner_count += 1

print(f"\n📊 Summary:")
print(f"   Total cells: {m * n}")
print(f"   Boundary cells (edges): {boundary_count}")
print(f"   Inner cells (potentially surrounded): {inner_count}")
print(f"\n💡 Water can only be trapped in the {inner_count} inner cells!")

📍 GRID WITH BOUNDARY MARKERS:
Legend: [B] = Boundary cell,  [I] = Inner cell

Row 0: [B]1  [B]4  [B]3  [B]1  [B]3  [B]2
Row 1: [B]3  [I]2  [I]1  [I]3  [I]2  [B]4
Row 2: [B]2  [B]3  [B]3  [B]2  [B]3  [B]1


📊 Summary:
   Total cells: 18
   Boundary cells (edges): 14
   Inner cells (potentially surrounded): 4

💡 Water can only be trapped in the 4 inner cells!


In [4]:
# What's actually stored in boundary_cells?
print("🔍 What's inside the boundary_cells list:")
print("=" * 70)
print("Format: (row, column, height)\n")

# Sort by height to show which will be processed first
boundary_cells_sorted = sorted(boundary_cells, key=lambda x: x[2])

for idx, (row, col, height) in enumerate(boundary_cells_sorted, 1):
    print(f"{idx:2d}. Position ({row},{col}) → height = {height}")

print("\n" + "=" * 70)
print("💡 The algorithm will process these in order from LOWEST to HIGHEST!")
print(f"   First processed: ({boundary_cells_sorted[0][0]},{boundary_cells_sorted[0][1]}) with height {boundary_cells_sorted[0][2]}")
print(f"   Last processed: ({boundary_cells_sorted[-1][0]},{boundary_cells_sorted[-1][1]}) with height {boundary_cells_sorted[-1][2]}")

🔍 What's inside the boundary_cells list:
Format: (row, column, height)

 1. Position (0,0) → height = 1
 2. Position (0,3) → height = 1
 3. Position (2,5) → height = 1
 4. Position (0,5) → height = 2
 5. Position (2,0) → height = 2
 6. Position (2,3) → height = 2
 7. Position (0,2) → height = 3
 8. Position (0,4) → height = 3
 9. Position (1,0) → height = 3
10. Position (2,1) → height = 3
11. Position (2,2) → height = 3
12. Position (2,4) → height = 3
13. Position (0,1) → height = 4
14. Position (1,5) → height = 4

💡 The algorithm will process these in order from LOWEST to HIGHEST!
   First processed: (0,0) with height 1
   Last processed: (1,5) with height 4


## 💧 Water Volume Calculation - The Key Idea!

### ✅ YES - We only calculate water at INNER cells!

But we need boundary cells to know **how high** the water can rise.

### The Formula:
```
For each inner cell:
    water_volume = max_height - cell_height
    
    where max_height = lowest boundary wall that water can escape through
```

### Example:
- Inner cell at (1,2) has height = **1**
- The lowest surrounding wall (boundary) = **2** (this is max_height)
- Water trapped = 2 - 1 = **1 unit** 💧

### Why we process boundary cells first:
- They define the "walls" that contain the water
- We work from outside → inside to find the minimum escape height
- This tells us how high water can rise at each inner cell

In [5]:
# Let's manually trace through the algorithm to see water calculation
heightMap = [[1,4,3,1,3,2],
             [3,2,1,3,2,4],
             [2,3,3,2,3,1]]

print("🔍 DETAILED WATER CALCULATION FOR EACH INNER CELL")
print("=" * 70)

# The 4 inner cells from our grid
inner_cells = [
    (1, 1, 2),  # Position (1,1), height = 2
    (1, 2, 1),  # Position (1,2), height = 1
    (1, 3, 3),  # Position (1,3), height = 3
    (1, 4, 2),  # Position (1,4), height = 2
]

print("\nInner cells to check:")
for row, col, height in inner_cells:
    print(f"  ({row},{col}): height = {height}")

print("\n" + "=" * 70)
print("💡 How the algorithm calculates water:\n")

# Simulate what happens (simplified)
# In reality, the algorithm processes these dynamically using the heap
# but the concept is the same

print("When we reach each inner cell from boundaries:")
print("-" * 70)

# Example calculations (these would be determined by the actual algorithm)
examples = [
    ((1,1), 2, 3, 1),  # cell, height, max_height_when_reached, water
    ((1,2), 1, 2, 1),  # This is a valley - traps water!
    ((1,3), 3, 3, 0),  # Too tall - no water
    ((1,4), 2, 3, 1),  # Traps water
]

total_water = 0
for (row, col), cell_height, max_height, water in examples:
    if water > 0:
        print(f"✓ Cell ({row},{col}): height={cell_height}, max_height={max_height}")
        print(f"  → Water = {max_height} - {cell_height} = {water} unit(s) 💧")
    else:
        print(f"✗ Cell ({row},{col}): height={cell_height}, max_height={max_height}")
        print(f"  → No water (cell too tall)")
    print()
    total_water += water

print("=" * 70)
print(f"🌊 TOTAL WATER TRAPPED: {total_water} units")
print("\nNote: The actual max_height values are determined by processing")
print("boundary cells from lowest to highest using the min-heap!")

🔍 DETAILED WATER CALCULATION FOR EACH INNER CELL

Inner cells to check:
  (1,1): height = 2
  (1,2): height = 1
  (1,3): height = 3
  (1,4): height = 2

💡 How the algorithm calculates water:

When we reach each inner cell from boundaries:
----------------------------------------------------------------------
✓ Cell (1,1): height=2, max_height=3
  → Water = 3 - 2 = 1 unit(s) 💧

✓ Cell (1,2): height=1, max_height=2
  → Water = 2 - 1 = 1 unit(s) 💧

✗ Cell (1,3): height=3, max_height=3
  → No water (cell too tall)

✓ Cell (1,4): height=2, max_height=3
  → Water = 3 - 2 = 1 unit(s) 💧

🌊 TOTAL WATER TRAPPED: 3 units

Note: The actual max_height values are determined by processing
boundary cells from lowest to highest using the min-heap!


## ✅ Summary: Your Understanding is Correct!

### 🎯 You're right:

**Q: We only care about inner cells?**  
✅ **YES!** Only inner cells can trap water

**Q: We calculate the volume at inner cells?**  
✅ **YES!** The formula is:
```
water_trapped = max_height - cell_height
```

### 🔑 The Complete Picture:

1. **Boundary cells** (14 cells) → Define the "walls"
   - We process these from **lowest to highest**
   - They tell us the **water level** (max_height)

2. **Inner cells** (4 cells) → Where water gets trapped
   - We calculate: `max_height - cell_height`
   - If result > 0 → water is trapped! 💧

3. **Total water** → Sum of all water trapped in inner cells
   - In this example: **4 units total** 🌊

### The Algorithm Flow:
```
1. Start with all 14 boundary cells in min-heap
2. Pop lowest boundary → update max_height
3. Check its neighbors (inner cells)
4. For each inner cell:
   IF cell_height < max_height:
      water += (max_height - cell_height)
5. Repeat until all cells visited
```

## 🎯 Why Process from LOWEST to HIGHEST? (Critical Insight!)

### ❌ Why NOT just process by position?

**The problem with processing by position:**
- Water finds the **LOWEST escape route**
- Position doesn't tell us which wall is weakest
- We'd get the wrong water level!

### ✅ Why process from LOWEST to HIGHEST:

**Think about real water:**
- Water always flows through the **LOWEST point** in the walls
- It doesn't care about the position of walls
- It only cares about **HEIGHT**

**Example scenario:**
```
Imagine a pool surrounded by walls:
- North wall: height 5
- South wall: height 5  
- East wall:  height 2 ← LOWEST!
- West wall:  height 5

Water level = 2 (flows out through east wall)
NOT 5! Position doesn't matter, only the LOWEST wall!
```

In [11]:
# Demonstration: Why lowest-first matters
heightMap = [[1,4,3,1,3,2],
             [3,2,1,3,2,4],
             [2,3,3,2,3,1]]

print("🔍 COMPARING TWO APPROACHES")
print("=" * 70)

# Look at inner cell (1,2) with height = 1
print("\n📍 Focus on inner cell (1,2) with height = 1")
print("-" * 70)

# What are its surrounding boundary cells?
neighbors = [
    ("top", (0, 2), 3),
    ("bottom", (2, 2), 3),
    ("left", (1, 1), 2),  # This is actually another inner cell!
    ("right", (1, 3), 3),
]

print("\nSurrounding cells of (1,2):")
for direction, pos, height in neighbors:
    print(f"  {direction:8s}: position {pos}, height = {height}")

print("\n" + "=" * 70)
print("❌ WRONG APPROACH: Process by position (e.g., top-to-bottom)")
print("-" * 70)
print("If we process top boundary first (height=3):")
print("  → We'd think water level = 3")
print("  → Water at (1,2) would be: 3 - 1 = 2 units")
print("  ❌ BUT WAIT! There's a lower wall we haven't checked yet!")

print("\n" + "=" * 70)
print("✅ CORRECT APPROACH: Process by HEIGHT (lowest-first)")
print("-" * 70)
print("Process boundaries from lowest to highest:")
print("  1. Find height=1 boundaries (if any)")
print("  2. Find height=2 boundaries")
print("  3. Find height=3 boundaries")
print("  ...")
print("\n  → We discover the LOWEST escape route first")
print("  → This gives us the TRUE water level")
print("  → Only then we calculate water correctly!")

print("\n" + "=" * 70)
print("💡 KEY INSIGHT:")
print("   Water escapes through the LOWEST wall, not the first wall we find!")
print("   So we MUST process from lowest to highest height!")

🔍 COMPARING TWO APPROACHES

📍 Focus on inner cell (1,2) with height = 1
----------------------------------------------------------------------

Surrounding cells of (1,2):
  top     : position (0, 2), height = 3
  bottom  : position (2, 2), height = 3
  left    : position (1, 1), height = 2
  right   : position (1, 3), height = 3

❌ WRONG APPROACH: Process by position (e.g., top-to-bottom)
----------------------------------------------------------------------
If we process top boundary first (height=3):
  → We'd think water level = 3
  → Water at (1,2) would be: 3 - 1 = 2 units
  ❌ BUT WAIT! There's a lower wall we haven't checked yet!

✅ CORRECT APPROACH: Process by HEIGHT (lowest-first)
----------------------------------------------------------------------
Process boundaries from lowest to highest:
  1. Find height=1 boundaries (if any)
  2. Find height=2 boundaries
  3. Find height=3 boundaries
  ...

  → We discover the LOWEST escape route first
  → This gives us the TRUE water lev

## 🌊 Real-World Analogy: The Dam Problem

### Imagine a container with walls of different heights:

```
        5ft          5ft
        ███          ███
        ███          ███
    2ft ███  WATER?  ███ 2ft  ← LOWEST wall!
    ███ ███  ~~~~    ███ ███
    ███ ███  ~~~~    ███ ███
    ━━━━━━━━━━━━━━━━━━━━━━━
```

**Question:** How high can the water level be?

**Answer:** **2 feet** - because water spills over the 2ft wall!

### The Algorithm Simulates This:

1. **Find the lowest wall first** (height = 2)
2. **That's your water level** (max_height = 2)
3. **Fill up to that level**
4. **Then continue** - look for next lowest wall, repeat

### If we processed by position instead:
- We might check the 5ft wall first
- Think water level = 5ft
- **WRONG!** Water already escaped through the 2ft wall!

### 🎯 The Min-Heap Ensures:
We **always** process the lowest wall first, guaranteeing we find the true water level!

In [12]:
# Concrete example: What happens when we process lowest-first
import heapq

heightMap = [[1,4,3,1,3,2],
             [3,2,1,3,2,4],
             [2,3,3,2,3,1]]

m, n = len(heightMap), len(heightMap[0])

print("🔍 STEP-BY-STEP: Why Lowest-First Matters")
print("=" * 70)

# Collect boundary cells
heap = []
for i in range(m):
    for j in range(n):
        if i == 0 or i == m-1 or j == 0 or j == n-1:
            heapq.heappush(heap, (heightMap[i][j], i, j))

print(f"\n📊 We have {len(heap)} boundary cells in the heap")
print("\nFirst 5 cells that will be processed (lowest height first):")
print("-" * 70)

# Show first 5 cells (without removing from heap)
temp_heap = heap.copy()
for i in range(min(5, len(temp_heap))):
    height, row, col = heapq.heappop(temp_heap)
    print(f"  {i+1}. Position ({row},{col}): height = {height}")

print("\n" + "=" * 70)
print("💡 OBSERVE:")
print("   - We start with height=1 cells (the WEAKEST walls)")
print("   - These define the initial water level")
print("   - As we move inward, max_height increases")
print("   - This ensures we NEVER overestimate the water level!")

print("\n" + "=" * 70)
print("🎯 CONCLUSION:")
print("   Processing by HEIGHT (not position) guarantees we find")
print("   the LOWEST escape route first → CORRECT water level! ✓")

🔍 STEP-BY-STEP: Why Lowest-First Matters

📊 We have 14 boundary cells in the heap

First 5 cells that will be processed (lowest height first):
----------------------------------------------------------------------
  1. Position (0,0): height = 1
  2. Position (0,3): height = 1
  3. Position (2,5): height = 1
  4. Position (0,5): height = 2
  5. Position (2,0): height = 2

💡 OBSERVE:
   - We start with height=1 cells (the WEAKEST walls)
   - These define the initial water level
   - As we move inward, max_height increases
   - This ensures we NEVER overestimate the water level!

🎯 CONCLUSION:
   Processing by HEIGHT (not position) guarantees we find
   the LOWEST escape route first → CORRECT water level! ✓


## 📚 Final Summary: Why Lowest-to-Highest is ESSENTIAL

### ❌ Processing by Position (Wrong!)
```
Problem: You don't know which wall is weakest
→ Might check a tall wall first
→ Think water level is high
→ But water already escaped through a shorter wall!
→ INCORRECT answer ❌
```

### ✅ Processing by Height (Correct!)
```
Solution: Always process lowest wall first
→ Check height=1 walls first
→ Then height=2 walls
→ Then height=3 walls, etc.
→ Guarantees we find the LOWEST escape point
→ CORRECT water level ✓
```

### 🔑 The Key Principle:

**"Water doesn't care about position. It only cares about finding the LOWEST way out!"**

### Why Min-Heap?
- Automatically sorts by height
- Always gives us the lowest unprocessed wall
- O(log n) operations - efficient!
- Perfect for this "always process lowest" requirement

### Real-World Example:
Imagine filling a bathtub with a crack in the side:
- It doesn't matter where the crack is (position)
- Only matters how HIGH the crack is
- Water fills up to crack level, then spills out
- Same concept here! 🛁💧